In [1]:
# Horray libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.linear_model import LogisticRegression

In [2]:
batch_size = 32
img_height = 180
img_width = 180

In [3]:
from src import make_test_train

In [4]:
test, train = make_test_train()

Found 5072 files belonging to 2 classes.
Found 502 files belonging to 2 classes.


In [5]:
# Data augmentation

data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)


In [6]:
preprocess_input = layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))

In [7]:
base_model = tf.keras.applications.ResNet50(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [8]:
def initialize_model():
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    base_learning_rate = 0.0001

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [9]:
model = initialize_model()

history = model.fit(
    train,
    validation_data=test,
    epochs=10
)
test_preds = model.predict(test)
# test_preds.to_csv(ideally with image names, definately with the actual labels)
train_preds = model.predict(train)
# train_preds.to_csv(ideally with image names, definately with the actual labels)

Epoch 1/10
159/159 [==============================] - 189s 1s/step - loss: 0.7177 - accuracy: 0.5138 - val_loss: 0.6886 - val_accuracy: 0.4801
Epoch 2/10
159/159 [==============================] - 196s 1s/step - loss: 0.6745 - accuracy: 0.5335 - val_loss: 0.6607 - val_accuracy: 0.4801
Epoch 3/10
159/159 [==============================] - 200s 1s/step - loss: 0.6438 - accuracy: 0.5576 - val_loss: 0.6206 - val_accuracy: 0.4801
Epoch 4/10
159/159 [==============================] - 193s 1s/step - loss: 0.6178 - accuracy: 0.5944 - val_loss: 0.5892 - val_accuracy: 0.4801
Epoch 5/10
159/159 [==============================] - 191s 1s/step - loss: 0.6033 - accuracy: 0.6215 - val_loss: 0.5743 - val_accuracy: 0.4801
Epoch 6/10
159/159 [==============================] - 139s 871ms/step - loss: 0.5845 - accuracy: 0.6431 - val_loss: 0.5606 - val_accuracy: 0.4801
Epoch 7/10
159/159 [==============================] - 136s 852ms/step - loss: 0.5726 - accuracy: 0.6656 - val_loss: 0.5431 - val_accuracy: 

In [12]:
test_labels = np.concatenate([y for x, y in test], axis=0)

In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

cm = confusion_matrix(test_labels, test_preds)
cm

ValueError: Classification metrics can't handle a mix of binary and continuous targets

model.predict(test.take(6))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()